#Install Faker
Pip install faker which is a pyhthon package that creates fake data based on the inputs you give. 

In [0]:
%pip install faker

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
from faker import Faker
import pandas as pd

fake = Faker()
data = [(fake.first_name(), fake.last_name(), fake.address(), fake.email(), fake.phone_number(), fake.date_of_birth()) for _ in range(50)]
df = pd.DataFrame(data, columns=['first_name', 'last_name', 'address', 'email', 'phone_number', 'date_of_birth'])
df['id'] = range(1, 51)

spark_df = spark.createDataFrame(df)
spark_df = spark_df.withColumn("id", monotonically_increasing_id())

spark_df.write.mode("overwrite").saveAsTable("main_sj.customers.customerdata")

In [0]:
%sql
Select * from main_sj.customers.customerdata

In [0]:
import random
import pandas as pd
from faker import Faker

fake = Faker()

# Generate fake data
data = [
    (fake.word(), round(random.uniform(10, 1000), 2), random.randint(1, 10), fake.date_this_decade())
    for _ in range(100)
]
orders_df = pd.DataFrame(data, columns=['article_name', 'unit_price', 'amount', 'purchase_date'])

# Calculate total_sum
orders_df['total_sum'] = orders_df['unit_price'] * orders_df['amount']

# Add customer_id from existing customer data
customer_ids = spark.sql("SELECT id FROM main_sj.customers.customerdata").rdd.flatMap(lambda x: x).collect()
orders_df['customer_id'] = [random.choice(customer_ids) for _ in range(100)]

# Add id column
orders_df['id'] = range(1, 101)

# Create Spark DataFrame
spark_orders_df = spark.createDataFrame(orders_df)

# Write to external table
spark_orders_df.write.mode("overwrite").saveAsTable("main_sj.customers.orders")

In [0]:
from pyspark.sql.functions import col, lit, rand
import random
import pandas as pd
from faker import Faker

fake= Faker()

data = [(i, fake.word(), round(random.uniform(10, 100), 2), random.randint(1, 10), 0.0, fake.date_this_year(), random.randint(1, 50)) for i in range(1, 101)]
df = pd.DataFrame(data, columns=['order_id', 'article_name', 'unit_price', 'amount', 'total_price', 'purchase_date', 'customer_id'])
df['total_price'] = df['unit_price'] * df['amount']

spark_df = spark.createDataFrame(df)

spark_df.write.mode("overwrite").saveAsTable("main_sj.customers.orders")